#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [2]:
loader = TextLoader("../DataIngestion/speech.txt")

In [7]:
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': '../DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of ri

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)

In [9]:
docs = text_splitter.split_documents(documents)

In [10]:
docs

[Document(metadata={'source': '../DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will b

In [13]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [14]:
db = FAISS.from_documents(docs, embeddings)

In [15]:
db

In [16]:
query="How does the speaker describe the desired outcome of the war?"
query_search = db.similarity_search(query)
query_search

[Document(id='2af8bbf2-96f6-49ce-8bb3-026dbad0c30e', metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ed79d8bd-05f9-4e83-bcae-0068a80b3b02', metadata={'source': '../DataIngestion/speech.txt'}, page_content='We have borne with their present government through all these bitter months becau

In [17]:
query_search[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [18]:
retriever = db.as_retriever()

In [19]:
query_search = retriever.invoke(query)
query_search

[Document(id='2af8bbf2-96f6-49ce-8bb3-026dbad0c30e', metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ed79d8bd-05f9-4e83-bcae-0068a80b3b02', metadata={'source': '../DataIngestion/speech.txt'}, page_content='We have borne with their present government through all these bitter months becau

In [20]:
query_search[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [21]:
search_and_score = db.similarity_search_with_score(query)
search_and_score

[(Document(id='2af8bbf2-96f6-49ce-8bb3-026dbad0c30e', metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(194.30635)),
 (Document(id='ed79d8bd-05f9-4e83-bcae-0068a80b3b02', metadata={'source': '../DataIngestion/speech.txt'}, page_content='We have borne with their present government through a

In [23]:
search_and_score[0][-1]

np.float32(194.30635)

In [24]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.5108685493469238,
 -0.39259058237075806,
 -0.2811056077480316,
 0.4476030766963959,
 -0.7402273416519165,
 -1.5125138759613037,
 -0.7012242674827576,
 0.3386024236679077,
 0.046869128942489624,
 -0.13584065437316895,
 0.2859903573989868,
 0.052440471947193146,
 -0.3365898132324219,
 0.5880857706069946,
 -0.3938499391078949,
 -0.4134729206562042,
 -0.7218430042266846,
 -0.20966652035713196,
 0.2558536231517792,
 0.04984397441148758,
 -0.034074604511260986,
 -0.40844160318374634,
 -0.6066627502441406,
 0.3827481269836426,
 -0.11300163716077805,
 0.2739143967628479,
 -0.5986611247062683,
 0.20387595891952515,
 1.132444977760315,
 -0.11244674026966095,
 -0.1725199818611145,
 0.5429068207740784,
 -0.04108256846666336,
 -0.6853285431861877,
 0.4342620372772217,
 -0.6314151287078857,
 0.4279518127441406,
 -1.9119406938552856,
 0.13529327511787415,
 -0.1590254306793213,
 0.25276124477386475,
 0.28936511278152466,
 0.5580092072486877,
 -0.4558134973049164,
 -1.4727994203567505,
 -0.282501250

In [25]:
len(embedding_vector)

1024

In [26]:
query_score = db.similarity_search_by_vector(embedding_vector)
query_score

[Document(id='2af8bbf2-96f6-49ce-8bb3-026dbad0c30e', metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ed79d8bd-05f9-4e83-bcae-0068a80b3b02', metadata={'source': '../DataIngestion/speech.txt'}, page_content='We have borne with their present government through all these bitter months becau

In [27]:
db.save_local("faiss_index")

In [28]:
db2 = db.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [29]:
db2.similarity_search(query)

[Document(id='2af8bbf2-96f6-49ce-8bb3-026dbad0c30e', metadata={'source': '../DataIngestion/speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='ed79d8bd-05f9-4e83-bcae-0068a80b3b02', metadata={'source': '../DataIngestion/speech.txt'}, page_content='We have borne with their present government through all these bitter months becau